<a href="https://colab.research.google.com/github/morgoth22a/AppFlowy/blob/main/qwen7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalacja Unsloth
!pip install --upgrade "unsloth[colab-new]"
# Alternatywna opcja, jeśli chcemy mieć pewność, że mamy najnowszą wersję
#!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install torch torchvision torchaudio
!pip install transformers peft bitsandbytes accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [3]:
from datasets import load_dataset

# Użycie opcji streaming=True, aby uniknąć ładowania całego 1TB zbioru do pamięci
# W tym przypadku, użyjemy wersji clean dataset, która jest bardziej odpowiednia
# do fine-tuningu.
dataset_name = "codeparrot/github-code-clean"
dataset = load_dataset(dataset_name, streaming=True, split="train", languages=["Python"])

# W przypadku, gdy chcemy użyć tylko kodu Pythona (zalecane)
# dataset = load_dataset(dataset_name, streaming=True, split="train", languages=["Python"])

README.md:   0%|          | 0.00/340 [00:00<?, ?B/s]

github-code-clean.py: 0.00B [00:00, ?B/s]

The repository for codeparrot/github-code-clean contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/github-code-clean.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [15]:
from transformers import AutoTokenizer

# Wczytanie tokenizera modelu Qwen 6.7B
model_id = "Qwen/CodeQwen1.5-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Maksymalna długość sekwencji. Wartość 2048 jest rekomendowana dla Google Colab.
# Dłuższe sekwencje mogą prowadzić do błędów OOM.
max_seq_length = 2048

# Funkcja do tokenizacji zbioru danych
def tokenize_function(examples):
    # Właściwy kod znajduje się w kolumnie 'content' lub 'code' w zależności od zbioru
    # Sprawdź strukturę danych przed użyciem.
    return tokenizer(examples['code'], truncation=True, max_length=max_seq_length)

# Aplikowanie tokenizacji w trybie strumieniowym
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [5]:
import torch
from unsloth import FastLanguageModel

# Model i jego parametry
model_id = "Qwen/CodeQwen1.5-7B-Chat"
max_seq_length = 2048
dtype = None # Opcjonalnie: torch.float16 dla BF16, jeśli wspierane
load_in_4bit = True # Aktywacja 4-bitowej kwantyzacji (QLoRA)

# Wczytanie modelu i tokenizera z Unsloth
# Użycie load_in_4bit=True włącza QLoRA, co redukuje zużycie VRAM o 70%
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

/tmp/ipython-input-3273246592.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.1: Fast Qwen2 patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [17]:
from peft import LoraConfig
from transformers import TrainingArguments
from unsloth import FastLanguageModel

# Ustawienie adapterów LoRA
# lora_r: Rank macierzy adapterów, wpływa na siłę adaptacji. Wyższe wartości = więcej parametrów.
# lora_alpha: Skalowanie wag adapterów.
# target_modules: Moduły (warstwy) modelu, do których zostaną dodane adaptery LoRA.

model = FastLanguageModel.get_peft_model(
    model = model, # Pass the base model
    r = 16, # LoRA attention dimension
    lora_alpha = 16, # Alpha parameter for LoRA scaling
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    lora_dropout = 0.05, # Dropout probability for LoRA layers
    bias = "none", # Bias type for LoRA
    use_gradient_checkpointing = "unsloth", # Recommended for memory efficiency
    random_state = 3407, # Random state for initialization
    use_rslora = False, # Whether to use RSLora
    loftq_config = None, # Optional LoFTQ configuration
)

# Definicja argumentów treningowych
training_args = TrainingArguments(
    output_dir="./fine_tuned_qwen_code",
    per_device_train_batch_size=4, # Zmniejsz w razie OOM
    gradient_accumulation_steps=4, # Zwiększ, aby symulować większy batch
    learning_rate=2e-4,
    num_train_epochs=3, # This will be ignored when max_steps is set
    max_steps=1000, # Specify the number of training steps for streaming dataset
    save_strategy="steps", # Save checkpoint every save_steps
    save_steps=500, # Save every 500 steps
    logging_steps=100,
    fp16=True, # Użyj precyzji FP16
    report_to="none", # Wyłącz raportowanie, jeśli nie jest potrzebne
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from trl import SFTTrainer

# Zdefiniowanie SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    packing=True, # Pakuje sekwencje, aby efektywniej wykorzystać GPU
    # dataset_text_field="code", # Usunięto, ponieważ użwamy tokenized_dataset z 'input_ids'
    args=training_args,
    max_seq_length=max_seq_length,
)

# Rozpoczęcie fine-tuningu
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 4}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16,000 | Num Epochs = 9,223,372,036,854,775,807 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 13,107,200 of 7,263,391,744 (0.18% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


In [ ]:
# Zapisz jedynie wytrenowane adaptery LoRA
# Plik będzie ważył tylko kilka MB
model.save_pretrained_peft("./qwen_code_adapter")

In [ ]:
from huggingface_hub import login

# Upewnij się, że masz token dostępu Hugging Face
# Możesz go wygenerować w ustawieniach swojego profilu
# W Google Colab można go bezpiecznie wprowadzić za pomocą secrets
login(token="YOUR_HUGGING_FACE_TOKEN")

# Zapisz i wgraj adapter na Hugging Face Hub
model.push_to_hub("your_username/fine-tuned-qwen-6.7b-code")
tokenizer.push_to_hub("your_username/fine-tuned-qwen-6.7b-code")

In [12]:
# Print the column names of the dataset to inspect its structure
print(dataset.column_names)

['code', 'repo_name', 'path', 'language', 'license', 'size']
